In [ ]:
!sudo apt install libspatialindex-dev

In [ ]:
!wget https://github.com/Inder-Dhillon/Toronto-Accidents-As-A-Network/blob/main/accident.py?raw=true
!mv accident.py\?raw\=true accident.py

--2020-10-31 17:38:46--  https://github.com/Inder-Dhillon/Toronto-Accidents-As-A-Network/blob/main/accident.py?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/Inder-Dhillon/Toronto-Accidents-As-A-Network/raw/main/accident.py [following]
--2020-10-31 17:38:47--  https://github.com/Inder-Dhillon/Toronto-Accidents-As-A-Network/raw/main/accident.py
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Inder-Dhillon/Toronto-Accidents-As-A-Network/main/accident.py [following]
--2020-10-31 17:38:47--  https://raw.githubusercontent.com/Inder-Dhillon/Toronto-Accidents-As-A-Network/main/accident.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.co

In [ ]:
pip install osmnx

     |████████████████████████████████| 92kB 2.7MB/s 
     |████████████████████████████████| 71kB 5.1MB/s 
     |████████████████████████████████| 11.6MB 8.5MB/s 
     |████████████████████████████████| 14.9MB 311kB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 972kB 37.7MB/s 
     |████████████████████████████████| 10.9MB 32.1MB/s 
     |████████████████████████████████| 14.8MB 313kB/s 
  Created wheel for Rtree: filename=Rtree-0.9.4-cp36-none-any.whl size=21767 sha256=360a489351e0fb3c2747beecf8c0bf45f4243e0e2c7719461281a7fa34fd1c4d
  Stored in directory: /root/.cache/pip/wheels/ff/20/c5/0004ef7acb96745ec99be960053902b0b414a2aa2dcad5834e
Successfully built Rtree
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.24.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement foli

In [ ]:
import osmnx as ox
import networkx as nx
import pandas as pd
import json
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from accident import ACCIDENT

In [ ]:
G = ox.graph_from_place('Toronto, Ontario, Canada', network_type='drive')

In [ ]:
ox.plot_graph(G, bgcolor='white', node_size=1, node_color="red", edge_linewidth=0.6, edge_alpha=1, dpi=1000, figsize=(20,20))

In [ ]:
print(nx.info(G))

In [ ]:
edges =  list(G.edges.data())
edges[160][2]

In [ ]:
nodes =  list(G.nodes.data())

## KSI Dataset:

In [ ]:
ksi_df = gpd.read_file("https://opendata.arcgis.com/datasets/cc17cc27ee5a4989b78d9a3810c6c007_0.geojson")

In [ ]:
ksi_df.head()

,Index_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS,District,WardNum,Division,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,Hood_ID,Neighbourhood,ObjectId,geometry
0,3389067,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,50 to 54,Major,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),1,POINT (-79.31880 43.69960)
1,3389068,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,15 to 19,Minor,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),2,POINT (-79.31880 43.69960)
2,3389069,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Driver,55 to 59,Minor,None,North,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),3,POINT (-79.31880 43.69960)
3,3389070,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,20 to 24,Minor,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),4,POINT (-79.31880 43.69960)
4,3389071,893184,2006,2006-01-01T05:00:00+00:00,236,2,WOODBINE AVE,O CONNOR DR,None,Major Arterial,Toronto and East York,19.0,54,43.699595,-79.318797,Intersection,Intersection Related,No Control,Clear,Dark,Wet,Non-Fatal Injury,Approaching,Passenger,15 to 19,Minor,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Yes,None,None,None,None,Yes,Yes,Yes,None,Yes,None,60,Woodbine-Lumsden (60),5,POINT (-79.31880 43.69960)


Combining Date and Hour to have one single column:

In [ ]:
ksi_df["DATE"] = pd.to_datetime(ksi_df['DATE']).dt.floor("D") + pd.to_timedelta(ksi_df["HOUR"], unit="hour")

Add datapoints to keep here:

In [ ]:
cols_to_keep = ["LATITUDE", "LONGITUDE", "ACCNUM", "DATE", "VISIBILITY", "LIGHT",	"RDSFCOND"]
ksi_df = ksi_df[cols_to_keep]

Counting fatalities using duplicate rows:

In [ ]:
fatalities = ksi_df["ACCNUM"].value_counts()
ksi_df["FATALITIES"] = ksi_df["ACCNUM"].apply(lambda x : fatalities[x])

In [ ]:
ksi_df = ksi_df.drop_duplicates()
ksi_df.reset_index(drop=True, inplace=True)

In [ ]:
ksi_df["G_NODE"]= ox.get_nearest_nodes(G, ksi_df["LONGITUDE"], ksi_df["LATITUDE"])
ksi_df.drop(["LATITUDE", "LONGITUDE"], axis=1, inplace=True)

In [ ]:
ksi_df = ksi_df.infer_objects()
ksi_df

,ACCNUM,DATE,VISIBILITY,LIGHT,RDSFCOND,FATALITIES,G_NODE
0,893184,2006-01-01 02:00:00+00:00,Clear,Dark,Wet,8,299173325
1,909646,2006-01-01 03:00:00+00:00,Snow,"Dark, artificial",Slush,5,299173325
2,884090,2006-01-02 07:00:00+00:00,Other,"Dark, artificial",Wet,2,299173325
3,885782,2006-01-04 19:00:00+00:00,Rain,Dark,Wet,2,299173325
4,882079,2006-01-06 22:00:00+00:00,Clear,"Dark, artificial",Dry,2,299173325
...,...,...,...,...,...,...,...
5685,9002468442,2019-12-23 06:00:00+00:00,Clear,Dawn,Wet,2,299173325
5686,9002471073,2019-12-23 14:00:00+00:00,Clear,Daylight,Dry,3,299173325
5687,9002481518,2019-12-25 08:00:00+00:00,Clear,Daylight,Wet,2,299173325
5688,9002496688,2019-12-27 23:00:00+00:00,Clear,"Dark, artificial",Dry,2,299173325


Creating column of accident objects:

In [ ]:
ksi_df["ACCIDENT"] = ksi_df.apply(lambda x: ACCIDENT(x["ACCNUM"], x["DATE"], x["VISIBILITY"], x["LIGHT"], x["RDSFCOND"], x["FATALITIES"]), axis=1)

In [ ]:
ksi_df

Collapsing dataset to get list of accidents for each node:

In [ ]:
ksi_df = ksi_df.groupby('G_NODE')['ACCIDENT'].apply(list).reset_index(name='ACCIDENTS')

In [ ]:
ksi_df

Converting dataset to dict to set node attributes:

In [ ]:
attr = ksi_df.set_index('G_NODE')['ACCIDENTS'].to_dict()

In [ ]:
nx.set_node_attributes(G, [], "accident_list")
nx.set_node_attributes(G, attr, "accident_list")

Example Nodes:

In [ ]:
G.nodes[1497793]

In [ ]:
G.nodes[158754118]